In [2]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import json 
from dotenv import load_dotenv
load_dotenv()
from urllib import parse


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from code_utils.utils import remove_too_long,type_score
from code_utils.BSO import update_bso_publications,get_bso_publications
from code_utils.pickle import load_cache


In [3]:
df_ipcc=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipcc_model.json')

In [5]:
df_ipcc=df_ipcc.rename(columns={'wg_chap':'ipcc'})

In [6]:
df_ipcc_group=df_ipcc.groupby(by=['doi'], dropna=True).agg({'ipcc': lambda x: list(x), 'countries': lambda x: list(x)})
df_ipcc_group=df_ipcc_group.reset_index()

In [7]:
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipcc_group.loc[:,'countries']=df_ipcc_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [8]:
df_ipbes=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\data_ipbes_visualization.json')

In [9]:
df_ipbes['ipbes']=df_ipbes.apply(lambda row: {'chapter': [x.replace('chap_','') for x in row['chap']]}, axis=1)

In [10]:
df_ipbes_group=df_ipbes.groupby(by=['doi'], dropna=True).agg({'ipbes': lambda x: list(x), 'countries': lambda x: list(x), 'freq': 'sum'})
df_ipbes_group=df_ipbes_group.reset_index()

In [11]:
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: x[0])
df_ipbes_group.loc[:,'countries']=df_ipbes_group.loc[:,'countries'].apply(lambda x: None if x==['None'] else x)

In [12]:
df_ipbes_group

,doi,ipbes,countries,freq
0,10.1001/archpedi.161.7.718,[{'chapter': ['5']}],None,1
1,10.1002/(sici)1099-0771(199612)9:4<231::aid-bd...,[{'chapter': ['4']}],[US],1
2,10.1002/16-0188.1,"[{'chapter': ['2.1', '5']}, {'chapter': ['2.1'...",[US],2
3,10.1002/2013ef000188,[{'chapter': ['2.1']}],[FR],1
4,10.1002/2013gb004654,[{'chapter': ['2.2']}],[DE],1
...,...,...,...,...
6575,10.7930/j0x63jt0,[{'chapter': ['glossary']}],None,1
6576,10.9755/ejfa.v25i1.15376,[{'chapter': ['3']}],[US],1
6577,10.9758/cpn.2015.13.3.239,[{'chapter': ['2.3']}],[TR],1
6578,10.9774/gleaf.978-1-907643-06-4_5,[{'chapter': ['2.1']}],None,1


In [13]:
df_ipcc_group

,doi,ipcc,countries
0,10.1001/jama.2012.108940,"[[{'name': 'wg2_chap_06', 'wg': '2', 'chap': 6...",[US]
1,10.1001/jama.2014.13186,"[[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7...","[GB, US]"
2,10.1001/jama.2014.15715,"[[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7...",[US]
3,10.1001/jama.2020.1097,"[[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7...",[US]
4,10.1001/jama.2020.15398,"[[{'name': 'wg2_chap_04', 'wg': '2', 'chap': 4...",[US]
...,...,...,...
54523,10.9774/gleaf.9781315651354,"[[{'name': 'wg3_chap_01', 'wg': '3', 'chap': 1...",[]
54524,10.9778/cmajo.20170047,"[[{'name': 'wg2_chap_14', 'wg': '2', 'chap': 1...",[CA]
54525,10.9783/9780812205381.291,"[[{'name': 'wg3_chap_14', 'wg': '3', 'chap': 1...",[]
54526,10.9783/9780812205381.336,"[[{'name': 'wg3_chap_14', 'wg': '3', 'chap': 1...",[]


In [14]:
df_ipcc_ipbes=pd.merge(df_ipcc_group,df_ipbes_group, on='doi', how='outer')
df_ipcc_ipbes

,doi,ipcc,countries_x,ipbes,countries_y,freq
0,10.1001/jama.2012.108940,"[[{'name': 'wg2_chap_06', 'wg': '2', 'chap': 6...",[US],NaN,NaN,NaN
1,10.1001/jama.2014.13186,"[[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7...","[GB, US]",NaN,NaN,NaN
2,10.1001/jama.2014.15715,"[[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7...",[US],NaN,NaN,NaN
3,10.1001/jama.2020.1097,"[[{'name': 'wg2_chap_07', 'wg': '2', 'chap': 7...",[US],NaN,NaN,NaN
4,10.1001/jama.2020.15398,"[[{'name': 'wg2_chap_04', 'wg': '2', 'chap': 4...",[US],NaN,NaN,NaN
...,...,...,...,...,...,...
59925,10.7930/j0x63jt0,NaN,NaN,[{'chapter': ['glossary']}],None,1.0
59926,10.9755/ejfa.v25i1.15376,NaN,NaN,[{'chapter': ['3']}],[US],1.0
59927,10.9758/cpn.2015.13.3.239,NaN,NaN,[{'chapter': ['2.3']}],[TR],1.0
59928,10.9774/gleaf.978-1-907643-06-4_5,NaN,NaN,[{'chapter': ['2.1']}],None,1.0


In [15]:
update_bso_publications()

2024-02-28 17:15:24,093 | code_utils.BSO | DEBUG | update bso publications files


DEBUG:code_utils.BSO:update bso publications files


KeyboardInterrupt: 

In [ ]:
bso=pd.DataFrame(get_bso_publications())

In [ ]:
df_bso=bso.transpose().reset_index()

In [9]:
df_bso.columns=['doi','rors', 'bso_local_affiliations']

NameError: name 'df_bso' is not defined

In [ ]:
df_bso

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes,df_bso,on='doi',how='left')

In [ ]:
def get_countries(row):
    if (isinstance(row.countries_x,list)==False)&(isinstance(row.countries_y,list)==False):
        return None
    elif (isinstance(row.countries_x,list))&(isinstance(row.countries_y,list)==False):
        return row.countries_x
    elif (isinstance(row.countries_x,list)==False)&(isinstance(row.countries_y,list)):
        return row.countries_y
    else:
        return row.countries_x

In [ ]:
df_ipcc_ipbes_bso['countries']=df_ipcc_ipbes_bso.apply(lambda row: get_countries(row),axis=1)

In [ ]:
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso[['doi', 'ipcc', 'ipbes', 'countries', 'rors', 'bso_local_affiliations']]

In [3]:
df_iso=pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\iso_countries.json')

In [5]:
df_iso=df_iso.rename(columns={'code':'countries'})

In [6]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_iso, on='countries', how='left')
del df_ipcc_ipbes_bso['countries']
df_ipcc_ipbes_bso=df_ipcc_ipbes_bso.rename(columns={'name':'countries'})

,name,countries
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS
...,...,...
238,Wallis and Futuna,WF
239,Western Sahara,EH
240,Yemen,YE
241,Zambia,ZM


In [ ]:
cached_openalex_data = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data.pkl')
cached_openalex_data_ipbes = load_cache({},module_path+'\\IPCC_bibliography\\AR6\\structured_data\\cached_openalex_data_ipbes.pkl')
cached_openalex_data.update(cached_openalex_data_ipbes)
df_cache=pd.DataFrame({'doi':list(cached_openalex_data.keys()),'OpenAlex_data':list(cached_openalex_data.values())})

In [ ]:
df_cache

In [ ]:
df_ipcc_ipbes_bso=pd.merge(df_ipcc_ipbes_bso,df_cache, on='doi', how='inner')

In [ ]:
dict_doi_bso=[]

In [ ]:
for i,row in df_ipcc_ipbes_bso.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    dict_doi_bso.append(dict_row2)

In [ ]:
len(dict_doi_bso)

In [ ]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)

In [ ]:
with open(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', 'w') as f:
      f.write(json_lines)

elastic

In [12]:
dd = pd.read_json(module_path+'\\IPCC_bibliography\\AR6\\structured_data\\bso_ipcc_ipbes.jsonl', lines=True)

In [22]:
dd2=dd.dropna(subset=['ipcc']).drop_duplicates(subset=['doi'])

In [25]:
len(dd2.loc[(dd2['ipcc'].apply(lambda wg_chap: any('1' in x.get('wg', '') for x in wg_chap)))])

12819

In [3]:
current_dict = dd.to_dict(orient='records')

In [4]:
list_bso_ipcc_ipbes=[]
for i in range(len(dd)):
    if len(current_dict[i]['OpenAlex_data'])==1:
        openalex_info = current_dict[i]['OpenAlex_data'][0]
    else:
        openalex_info = current_dict[i]['OpenAlex_data'][1]
    del current_dict[i]['OpenAlex_data']
    for f in openalex_info:
        current_dict[i]['openalex_'+f] = openalex_info[f]  

In [5]:
df_elastic_input=pd.DataFrame(current_dict)

In [6]:
df_elastic_input.loc[:,'openalex_concepts']=df_elastic_input.loc[:,'openalex_concepts'].apply(type_score)

In [7]:
del df_elastic_input['openalex_abstract_inverted_index']

In [8]:
len(df_elastic_input)

58520

In [9]:
dict_doi_bso=[]
for i,row in df_elastic_input.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    #dict_row3=remove_too_long(dict_row2)
    dict_doi_bso.append(dict_row2)

In [10]:
json_lines='\n'.join(json.dumps(entry) for entry in dict_doi_bso)
with open('bso_ipcc_ipbes_elastic.jsonl', 'w') as f:
      f.write(json_lines)

In [11]:
#df=pd.read_json('bso_ipcc_ipbes_elastic.jsonl', lines=True)

In [ ]:
es_url_without_http = os.getenv("ES_URL").replace('https://','').replace('http://','')
es_host = f'https://{os.getenv("ES_LOGIN_TEDS_BACK")}:{parse.quote(os.getenv("ES_PASSWORD_TEDS_BACK"))}@{es_url_without_http}'
index_name='teds-bibliography'